# In this notebook we benchmark the performance of Sinkhon vs damped Newton for different distance functions.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/DampedNewtonVsSinkbhorn_for diff_norm_images'):
    os.makedirs('../Images/DampedNewtonVsSinkbhorn_for diff_norm_images')

In [ ]:
""" Different norms"""
def p_norms( x, y, p ):
   # L-1 norm
   if p == 1:
      return  abs( np.sum( x, 0 )[:,None] - np.sum( y, 0 )[None,:] )  
   # L-2 norm
   elif p == 2:
      return  np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )
   # L-4 norm
   elif p == 4:
    return np.sum( x**4, 0 )[:,None] + np.sum( y**4, 0 )[None,:] + 4 * ( x**3 ).transpose().dot( y ) - 4 * x.transpose().dot( y**3 ) + 6 * ( x**2 ).transpose().dot( y**2 )
   # L-infty norm
   elif p == 'inf':
      return np.maximum( abs( x[0,:][:,None] - y[0,:][None,:] ), abs( x[1,:][:,None] - y[1:][None:] ) )

"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
import computational_OT

In [ ]:
def spectral_decomposition( mat ):
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort( eig ) 
    eig = eig[ sorting_indices ]
    v   = v[ : , sorting_indices ]    
    print( "List of smallest eigenvalues: ", eig[ : 10 ] )
    print( "List of largest  eigenvalues: ", eig[ - 10 : ] )
    return eig, v

In [ ]:
def build_preconditioners( num_eigs, modified_Hessian, N, ansatz = True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  =  eigenvalues[ sorting_indices ]
    eigenvectors = eigenvectors[ :,  sorting_indices ]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[ : , 0 ]
    else:
        null_vector = np.hstack( ( np.ones(N[0]), - np.ones(N[1]) ) )
        norm = np.sqrt( N[0] + N[1] )
        null_vector = null_vector/norm
    # Form other vectors (only 13)
    _,m = eigenvectors.shape
    indices=[]
    for i in range(num_eigs//2):
        indices.append( m - i - 2 )
        indices.append( i + 1 )
    # end for
    if num_eigs%2 !=0:
        indices.append( m - 1 - ( num_eigs//2 ) )
   
    precond_vectors = eigenvectors[:, indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    # end for
    return null_vector, precond_vectors

In [ ]:
N = [ 1000, 1200 ]
epsilons = [ 0.1, 0.09, 0.05, 0.03 ] 


In [ ]:
x = np.random.rand( 2,N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r = 0.8 + .2 * np.random.rand( 1, N[1] )
y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )

# I. L-1 norm

###  Sinkhorn

In [ ]:
# Sinkhorn
print( "Sinkhorn... ")
print( "Doing for (",N[0], N[1],")." )
SinkhornP_L_1 = []
results_Sinkhorn_L_1 = []
times_Sinkhorn_L_1 = []
#Cost matrix
C = p_norms( x, y, 2 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print("For epsilon = "+str(eps)+":")       
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 10000 )
  results_Sinkhorn_L_1.append( out )
  end = time.time()
  times_Sinkhorn_L_1.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP_L_1.append( P_opt )
# end for 

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn_L_1) ):
  error = np.asarray( results_Sinkhorn_L_1[i]['error_a'] ) + np.asarray( results_Sinkhorn_L_1[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( loc = 'upper right' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ConvergenceSinkhornvaryingepsilonL1.pdf", format = 'pdf' )
plt.show()

###  Damped Newton


In [ ]:
# Damped Newton
print( "Damped Newton... " )
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L_1 = []
results_dampedNewton_L_1  = []
times_dampedNewton_L_1    = []
Hessians_dampedNewton_L_1 = []
#Cost matrix
C = p_norms( x, y, 1 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( -C/eps )
    f, g = a, b
    print( " |- Iterating")  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L_1.append( out )
        times_dampedNewton_L_1.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L_1.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L_1.append( mat )
    else:
        epsilons.remove( eps )
# end for

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton_L_1[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/eigenhistunstabilizedL1.pdf", format = 'pdf')
plt.show()

### Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton_L_1[-1], N, ansatz = False )

In [ ]:
# Damped Newton with preconditioning
print( "Damped Newton with preconditioning and iterative inversion... " )
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_L_1 = []
dampedNewtonwithprecondP_L_1 = []
results_dampedNewtonwithprecond_L_1  = []
times_dampedNewtonwithprecond_L_1    = []
# Cost matrix
C = p_norms( x, y, 1 )
# a and b
a = normalize( np.ones(N[0]) )
b = normalize( np.ones(N[1]) )
f, g = None, None
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType= 'cg' )
    results_dampedNewtonwithprecond_L_1.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_L_1.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_L_1.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_L_1.append( Optimizer.modified_Hessian )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_L_1) ):
  error = np.asarray( results_dampedNewtonwithprecond_L_1[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_L_1[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ErrorLinesearchNewton_final_cgL1.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# II. L-2 norm

## Sinkhorn

In [ ]:
# Sinkhorn
print( "Sinkhorn... " )
print("Doing for (",N[0], N[1],").")
SinkhornP_L_2 = []
results_Sinkhorn_L_2 = []
times_Sinkhorn_L_2 = []
#Cost matrix
C = p_norms( x, y, 2 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print("For epsilon = "+str(eps)+":")       
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 10000 )
  results_Sinkhorn_L_2.append( out )
  end = time.time()
  times_Sinkhorn_L_2.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP_L_2.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn_L_2) ):
  error = np.asarray( results_Sinkhorn_L_2[i]['error_a'] ) + np.asarray( results_Sinkhorn_L_2[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ConvergenceSinkhornvaryingepsilonL2.pdf", format = 'pdf' )
plt.show()

## Damped Newton without preconditioning


In [ ]:
# Damped Newton with preconditioning
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L_2 = []
results_dampedNewton_L_2  = []
times_dampedNewton_L_2    = []
Hessians_dampedNewton_L_2 = []
#Cost matrix
C = p_norms( x, y, 2 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L_2.append( out )
        times_dampedNewton_L_2.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L_2.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L_2.append( mat )
    else:
        epsilons.remove( eps )
# end for

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton_L_2[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/eigenhistunstabilizedL2.pdf", format = 'pdf' )
plt.show()

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton_L_2[-1], N, ansatz = False )

## Damped Newton with preconditioning

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton with preconditioning and iterative inversion... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_L_2 = []
dampedNewtonwithprecondP_L_2 = []
results_dampedNewtonwithprecond_L_2  = []
times_dampedNewtonwithprecond_L_2    = []
# Cost matrix
C = p_norms( x, y, 2 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
f, g = None, None
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_L_2.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_L_2.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_L_2.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_L_2.append( Optimizer.modified_Hessian )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_L_2) ):
  error = np.asarray( results_dampedNewtonwithprecond_L_2[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_L_2[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ErrorLinesearchNewton_final_cgL2.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# III. L4-norm

## Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn... ")
print("Doing for (",N[0], N[1],").")
SinkhornP_L_4 = []
results_Sinkhorn_L_4 = []
times_Sinkhorn_L_4 = []
#Cost matrix
C = p_norms( x, y, 4 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print("For epsilon = "+str(eps)+":")       
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 10000 )
  results_Sinkhorn_L_4.append( out )
  end = time.time()
  times_Sinkhorn_L_4.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP_L_4.append( P_opt )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn_L_4) ):
  error = np.asarray( results_Sinkhorn_L_4[i]['error_a'] ) + np.asarray( results_Sinkhorn_L_4[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ConvergenceSinkhornvaryingepsilonL4.pdf", format = 'pdf' )
plt.show()

## Damped Newton without preconditioning

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L_4 = []
results_dampedNewton_L_4  = []
times_dampedNewton_L_4    = []
Hessians_dampedNewton_L_4 = []
#Cost matrix
C = p_norms( x, y, 4 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L_4.append( out )
        times_dampedNewton_L_4.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L_4.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L_4.append( mat )
    else:
        epsilons.remove( eps )
# end for

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton_L_4[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    if not np.isnan(eigs[i]).any():
        ax[i].hist( eigs[i], 50 )
        ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
        ax[i].set_xlabel( "Eigenvalues" )
        ax[i].set_yscale( "log" )    
    else:
        print("The eigenvalues for "+str(epsilons[i])+ " contains Nans.")
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/eigenhistunstabilizedL4.pdf", format = 'pdf' )
plt.show()

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton_L_4[-1], N, ansatz = False )

## Damped Newton with preconditioning

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton with preconditioning and iterative inversion... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_L_4 = []
dampedNewtonwithprecondP_L_4 = []
results_dampedNewtonwithprecond_L_4 = []
times_dampedNewtonwithprecond_L_4    = []
# Cost matrix
C = p_norms( x, y, 4 )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
f, g = None, None
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps ) 
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_L_4.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_L_4.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_L_4.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_L_4.append( Optimizer.modified_Hessian )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_L_4) ):
  error = np.asarray( results_dampedNewtonwithprecond_L_4[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_L_4[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ErrorLinesearchNewton_final_cgL4.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# IV. L-infinity

## Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn... ")
print("Doing for (",N[0], N[1],").")
SinkhornP_L_infty = []
results_Sinkhorn_L_infty = []
times_Sinkhorn_L_infty = []
#Cost matrix
C = p_norms( x, y, 'inf' )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print("For epsilon = "+str(eps)+":")       
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 10000 )
  results_Sinkhorn_L_infty.append( out )
  end = time.time()
  times_Sinkhorn_L_infty.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP_L_infty.append( P_opt )
  SinkhornP_L_infty.append( GetP( np.exp( out['potential_f']/eps ), K, np.exp( out['potential_g']/eps ) ) )  
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 )  )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn_L_infty) ):
  error = np.asarray( results_Sinkhorn_L_infty[i]['error_a'] ) + np.asarray( results_Sinkhorn_L_infty[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ConvergenceSinkhornvaryingepsilonLinfty.pdf", format = 'pdf' )
plt.show()

## Damped Newton without preconditioning

In [ ]:
# Damped Newton
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_L_infty = []
results_dampedNewton_L_infty  = []
times_dampedNewton_L_infty    = []
Hessians_dampedNewton_L_infty = []
#Cost matrix
C = p_norms( x, y, 'inf' )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton(  K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_L_infty.append( out )
        times_dampedNewton_L_infty.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_L_infty.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_L_infty.append( mat )
    else:
        epsilons.remove( eps )
# end for

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton_L_infty[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/eigenhistunstabilizedLinfty.pdf", format = 'pdf' )
plt.show()

## Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton_L_infty[-1], N, ansatz = False )

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton with preconditioning and iterative inversion... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_L_infty = []
dampedNewtonwithprecondP_L_infty = []
results_dampedNewtonwithprecond_L_infty  = []
times_dampedNewtonwithprecond_L_infty    = []
# Cost matrix
C = p_norms( x, y, 'inf' )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
f, g = None, None
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_L_infty.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_L_infty.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_L_infty.append( P_opt )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_L_infty.append( Optimizer.modified_Hessian )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_L_infty) ):
  error = np.asarray( results_dampedNewtonwithprecond_L_infty[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_L_infty[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ErrorLinesearchNewton_final_cgLinfty.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )

# V. 1-cosine similarity

## Sinkhorn

In [ ]:
# Sinkhorn
print("Sinkhorn... ")
print("Doing for (",N[0], N[1],").")
SinkhornP_cosine = []
results_Sinkhorn_cosine = []
times_Sinkhorn_cosine = []
#Cost matrix
C = 1 - ( x.transpose().dot( y ) )/( np.linalg.norm( x, 2 ) * np.linalg.norm( y, 2 ) )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  #Kernel
  K = np.exp( - C/eps )
  print("For epsilon = "+str(eps)+":")       
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 10000 )
  results_Sinkhorn_cosine.append( out )
  end = time.time()
  times_Sinkhorn_cosine.append( end - start  )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP_cosine.append( P_opt )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn_cosine) ):
  error = np.asarray( results_Sinkhorn_cosine[i]['error_a'] ) + np.asarray( results_Sinkhorn_cosine[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend()
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ConvergenceSinkhornvaryingepsilonLcosine.pdf", format = 'pdf' )
plt.show()

## Damped Newton without preconditioning

In [ ]:
# Damped Newton 
print("Damped Newton... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
dampedNewtonP_cosine = []
results_dampedNewton_cosine  = []
times_dampedNewton_cosine    = []
Hessians_dampedNewton_cosine = []
#Cost matrix
C = 1 - ( x.transpose().dot( y ) )/( np.linalg.norm( x, 2 ) * np.linalg.norm( y, 2 ) )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton( K,
                                                a,
                                                b,
                                                f,
                                                g,
                                                eps,
                                                rho,
                                                c )
    out = Optimizer._update(    max_iterations = 50,
                                debug = False )
    end = time.time()
    if out != -1:
        results_dampedNewton_cosine.append( out )
        times_dampedNewton_cosine.append( end - start )
        print( " |- Computing P" )
        print( "" )
        u_opt = np.exp( out['potential_f']/eps )
        K = np.exp( - C/eps )
        v_opt =  np.exp( out['potential_g']/eps )
        P_opt = GetP( u_opt, K, v_opt )
        dampedNewtonP_cosine.append( P_opt )
        print( " |- Recording (unstabilized) Hessian \n" )
        mat  = - eps * Optimizer.Hessian
        diag = 1/np.sqrt( np.concatenate( ( a, b ), axis = None ) )
        mat = diag[:,None] * mat * diag[None,:]
        Hessians_dampedNewton_cosine.append( mat )
    else:
        epsilons.remove( eps )
# end for

In [ ]:
eigs = []
eigvecs = []
for i in range( len(epsilons) ) :
    eps = epsilons[i]
    print( "Spectral statistics of Hessian for epsilon = "+str(eps) )
    ev = spectral_decomposition( Hessians_dampedNewton_cosine[i] )
    eigs.append( ev[0] )
    eigvecs.append( ev[1] )
    print("")
# end for

In [ ]:
fig, ax=plt.subplots( figsize = ( 5, 12 ), nrows = len(epsilons), ncols = 1, sharey = True )
plt.title( "Histogram of eigenvalues." )
for i in range( len(epsilons) ):
    ax[i].hist( eigs[i], 50 )
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]) )
    ax[i].set_xlabel( "Eigenvalues" )
    ax[i].set_yscale( "log" )
# end for
plt.subplots_adjust( wspace = 0, hspace = 0.5 )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/eigenhistunstabilizedLcosine.pdf", format = 'pdf' )
plt.show()

## Damped Newton with preconditioning

In [ ]:
num_eigs = 10
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_dampedNewton_cosine[-1], N, ansatz = False )

In [ ]:
# Damped Newton with preconditioning
print("Damped Newton with preconditioning and iterative inversion... ")
print( "Doing for (",N[0], N[1],")." )
rho = 0.95
c = 0.05
reset_starting_point = True
final_modified_Hessians_cosine = []
dampedNewtonwithprecondP_cosine = []
results_dampedNewtonwithprecond_cosine  = []
times_dampedNewtonwithprecond_cosine    = []
# Cost matrix
C = 1 - ( x.transpose().dot( y ) )/( np.linalg.norm( x, 2 ) * np.linalg.norm( y, 2 ) )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
f, g = None, None
for eps in epsilons:
    print( "For epsilon = "+str(eps)+":" )    
    #Kernel
    K = np.exp( - C/eps )
    if (f is None) or (g is None): 
        f, g = a, b
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.damped_Newton_with_preconditioning(    K,
                                                                        a,
                                                                        b,
                                                                        f,
                                                                        g,
                                                                        eps,
                                                                        rho,
                                                                        c,
                                                                        null_vector,
                                                                        precond_vectors[:] )
    out = Optimizer._update(    max_iterations = 50,
                                iterative_inversion = 30,
                                version = None,
                                debug = False,
                                optType = 'cg' )
    results_dampedNewtonwithprecond_cosine.append( out )
    end = time.time()
    times_dampedNewtonwithprecond_cosine.append( end - start )
    print( " |- Computing P" )
    print( "" )
    u_opt = np.exp( out['potential_f']/eps )
    K = np.exp( - C/eps )
    v_opt =  np.exp( out['potential_g']/eps )
    P_opt = GetP( u_opt, K, v_opt )
    dampedNewtonwithprecondP_cosine.append( P_opt ) 
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
    final_modified_Hessians_cosine.append( Optimizer.modified_Hessian )
# end for

### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )
for i in range( len(results_dampedNewtonwithprecond_cosine) ):
  error = np.asarray( results_dampedNewtonwithprecond_cosine[i]['error_a'] ) + np.asarray( results_dampedNewtonwithprecond_cosine[i]['error_b'] )
  plt.plot( error, label = 'Damped Newton with preconditioning for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.xlabel( "Number of iterations" )
plt.ylabel( "Error in log-scale" )
plt.legend()
plt.yscale( 'log' )
plt.savefig( "../Images/DampedNewtonVsSinkbhorn_for diff_norm_images/ErrorLinesearchNewton_final_cgLcosine.pdf", format = 'pdf' )
plt.show()
print( "\n Error plots can increase! The error is not the objective function!" )